In [1]:
import os
import sqlite3
import csv
from pythainlp.tag.thainer import ThaiNameTagger

In [2]:
cur_dir = globals()['_dh'][0]

In [3]:
def find_named_entities(text):
    ner = ThaiNameTagger()
    nes = ner.get_ner(text) 
    named_entities = []
    tag_focus = ["B-LOCATION", "I-LOCATION", "B-PERSON", "I-PERSON", "B-ORGANIZATION", "I-ORGANIZATION"]
    l = ""
    # print(nes)
    for idx, ne in enumerate(nes):
        if ne[2] in tag_focus and ne[1]:
            if idx+1 < len(nes):
                if nes[idx+1][2] in tag_focus and nes[idx+1][0] != " ":
                    l += ne[0]
                else:
                    l += ne[0]
                    named_entities.append(l.strip())
                    l = ""
            else:
                l += ne[0]
                named_entities.append(l.strip())
                l = ""
    return named_entities

In [10]:
# function to Initialize database
def populate_db():
    cur_dir = globals()['_dh'][0]
    db_file = cur_dir + "/locationdata.db"
    con = sqlite3.connect(db_file)
    cur = con.cursor()
    cur.execute("DROP TABLE IF EXISTS locations")
    
    cur.execute("CREATE TABLE locations(province TEXT, district TEXT, subdistrict TEXT, zipcode TEXT, latitude TEXT, longitude TEXT)")
        
    with open(str(cur_dir)+ "/data/locations.csv", encoding = 'UTF') as info:
        reader = csv.reader(info)
        for row in reader:
            cur.execute("INSERT INTO locations VALUES(?, ?, ?, ?, ?, ?)", row)
                
        con.commit()
    return con
        
def db_has_data(con):
    cur = con.cursor()
    cur.execute("SELECT Count(*) FROM sqlite_master WHERE name='locations';")
    data = cur.fetchone()[0]
        
    if data > 0:
        cur.execute("SELECT Count(*) FROM locations")
        data = cur.fetchone()[0]
        return data > 0 
        
    return False

In [11]:
con = populate_db()

In [12]:
def find_subdistrict(named_entities):
    if len(named_entities) > 0:
        for idx, ent in enumerate(named_entities):
            if 'ต.' in ent:
                cur = con.cursor()
                cur.execute("SELECT subdistrict FROM locations WHERE subdistrict = '" +ent.replace("ต.","")+ "'")
                res = cur.fetchone()
                if(res is not None):
                    return res[0]
            
        cur = con.cursor()
        cur.execute("SELECT subdistrict FROM locations WHERE subdistrict IN (" + ",".join("?"*len(named_entities)) + ")", [p.lower() for p in named_entities])
        res = cur.fetchone()
        if res is not None:
            return res[0]
        return None
               
                    
        

In [13]:
def find_district(named_entities):
    if len(named_entities) > 0:
        for idx, ent in enumerate(named_entities):
            if 'อ.' in ent:
                cur = con.cursor()
                cur.execute("SELECT district FROM locations WHERE district = '" +ent.replace("อ.","")+ "'")
                res = cur.fetchone()
                if(res is not None):
                    return res[0]
            
        cur = con.cursor()
        cur.execute("SELECT district FROM locations WHERE district IN (" + ",".join("?"*len(named_entities)) + ")", [p.lower() for p in named_entities])
        res = cur.fetchone()
        if res is not None:
            return res[0]
        return None
               

In [14]:
def find_province(named_entities):
    if len(named_entities) > 0:
        for idx, ent in enumerate(named_entities):
            if 'จ.' in ent:
                cur = con.cursor()
                cur.execute("SELECT province FROM locations WHERE province = '" +ent.replace("จ.","")+ "'")
                res = cur.fetchone()
                if(res is not None):
                    return res[0]
            
        cur = con.cursor()
        cur.execute("SELECT province FROM locations WHERE province IN (" + ",".join("?"*len(named_entities)) + ")", [p.lower() for p in named_entities])
        res = cur.fetchone()
        if res is not None:
            return res[0]
        return None

In [15]:
loc = find_location("หญิงวัย 61 ปี เจ้าของร้านชำที่ไพศาลี นครสวรรค์ ตกเป็นเหยื่อกระสุนอดีตลูกเขยโหด ใช้ปืนลูกซองยาวเข้ามายิงตายคาร้าน ตำรวจตามรวบได้ทันควัน สารภาพแค้นที่ผู้ตายไม่เห็นคุณค่า กีดขวางไม่ให้อยู่กินกับลูกสาว ทั้งที่พยายามพิสูจน์ตัวเองมานับ 10 ปี เวลา 09.00 น. วันที่ 12 มิ.ย. ร.ต.อ.ภาสกร เรืองศรีธัญกิจ รอง สว.(สอบสวน) สภ.ไพศาลี จ.นครสวรรค์ ได้รับแจ้งมีเหตุยิงกันตายที่ร้านค้าของชำในตลาดไพศาลี จึงพร้อมด้วย")
print(loc)

{'district': 'ไพศาลี', 'subdistrict': 'ไพศาลี', 'province': 'นครสวรรค์', 'latitude': '15.637', 'longitude': '100.700'}


In [16]:
# con = populate_db()
def find_location(text):
    words = find_named_entities(text)
    # print(words)
    district = find_district(words)
    subdistrict = find_subdistrict(words)
    province = find_province(words)
    
    location = {
        "district" : district,
        "subdistrict" : subdistrict,
        "province": province
    }
    
    if subdistrict and district:
        cur = con.cursor()
        cur.execute("SELECT latitude, longitude FROM locations WHERE subdistrict = '" +subdistrict+ "'" + "and district = '" + district + "'")
        res = cur.fetchone()
        if res is not None:
            location['latitude'] = res[0]
            location['longitude'] = res[1]
    elif subdistrict:
        cur = con.cursor()
        cur.execute("SELECT district, latitude, longitude FROM locations WHERE subdistrict = '" +subdistrict+ "'")
        res = cur.fetchone()
        if res is not None:
            location['district'] = res[0]
            location['latitude'] = res[1]
            location['longitude'] = res[2]
    else:
        location['latitude'] = None
        location['longitude'] = None
    return location
    
   
        
        

In [17]:
loc = find_location("เพื่อปกปิดการตาย&quot; จับกุมได้ที่บริเวณริมถนนสาธารณะ สายถ้ำโกบ-เขาดิน หมู่ 3 ต.หน้าเขา อ.เขาพนม จ.กระบี่ สืบ")
print(loc)

{'district': 'เขาพนม', 'subdistrict': 'หน้าเขา', 'province': 'กระบี่', 'latitude': '8.343', 'longitude': '98.966'}


In [294]:
loc = find_location("หรือ นายธีรเดช ขจรสุข อายุ 50 ปี ผู้ต้องหาตามหมายจับศาลจังหวัดเชียงใหม่ ที่ 716/2559 ลงวันที่ 5 ต.ค.2559 ในฐานความผิด กระทำชำเราเด็กอายุไม่เกิน 15 ปี ,กระทำชำเราเด็กอายุเกิน 15 ปี, กระทำอนาจารเด็กอายุไม่เกิน 15 ปี")
print(loc)

{'district': None, 'subdistrict': None, 'province': None, 'latitude': None, 'longitude': None}


In [298]:
loc = find_location("โดยจับกุมได้ที่วัดสายวารี ต.บ้านเสด็จ อ.เคียนซา จ.สุราษฎร์ธานี พ.ต.อ.มารุต กล่าวว่า")
print(loc)

{'district': 'เคียนซา', 'subdistrict': 'บ้านเสด็จ', 'province': 'สุราษฎร์ธานี', 'latitude': '8.736', 'longitude': '99.024'}


In [18]:
# df = pd.read_csv('data/sentiment_analysis_data.csv')
# sent_loc = []
# for index, row in df.iterrows():
#     sent_loc.append(find_location(row['text']))
# df['location'] = sent_loc

In [19]:
# df.to_csv('data/result.csv')  

In [20]:
# df = pd.read_csv('data/cls/sentence_crimelocation.csv')
# sent_loc = []
# for index, row in df.iterrows():
#     sent_loc.append(find_location(row['text']))
# df['location'] = sent_loc

In [21]:
# df.to_csv('data/cls/result.csv')  